In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -q /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip 
!unzip -q /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
# 1. 이미지를 그림으로, PIL에서 Image.open()

from PIL import Image   #Python Image Library

os.listdir("train")

Image.open("train/cat.7685.jpg")

In [ ]:
# 데이터 전처리

# 1.df만들기, 칼럼2개 - (각각 이미지의 경로, 정답)

train = pd.DataFrame({"path":os.listdir("train")})

train["path"] = "train/"+train["path"]
train["ans"] = train["path"].apply(lambda x : x.split("/")[1].split(".")[0])

train

In [ ]:
# 이미지 전처리
from keras.preprocessing.image import ImageDataGenerator

idg = ImageDataGenerator()

train_generator = idg.flow_from_dataframe(train, x_col = "path", y_col = "ans", batch_size = 128, target_size = (100,100))
# 데이터셋 이름, x경로, y정답값, batch_size, 고정할 사진size

# 각각의 jpg파일은 픽셀크기가 다를 수 있으므로
# ImageDataGenerator()함수로 모든 이미지의 크기를 target_size()로 재조정한다. 


In [ ]:
from keras import Sequential
from keras.layers import *

model = Sequential()

model.add(Conv2D(32, (3,3), activation = "relu", input_shape = (100,100,3)))                                # Conv2D() : convolution layer로 학습
# filter옵션 : 얼마나 많은 특징 추출할 것인가, 스스로 가장 잘 나누는 특징을 추출한다
# kernel_szie옵션 : 숫자를 9개씩 sliding, 일반적으로 3x3이 가장 효율적
# activation옵션 : "relu", gradient를 update하면서
# input_shape : 100,100의 사이즈의, 3개의 RGB채널로 나눔

model.add(Flatten()) # 1차원으로 펼치기

model.add(Dense(2, activation = "softmax"))

model.compile(metrics = ["acc"], loss = "categorical_crossentropy", optimizer = "adam")

model.fit(train_generator)
# 이미지 학습에서는 많은 이미지들을 한번에 학습시킬 수 없으므로
# 제너레이터를 만들어 끊어서 학습시킨다(128번씩), 제너레이터에는 정답값도 이미 들어가있다.


In [ ]:
os.listdir("test")

test = pd.DataFrame({"path":os.listdir("test")})

test["path"] = "test/"+test["path"]
test

idg2 = ImageDataGenerator()

test_generator = idg2.flow_from_dataframe(test, x_col = "path", y_col = None, batch_size = 128, target_size = (100,100), class_mode = None, shuffle = False)
# test제너레이터에는 옵션 2개 추가
#  class_mode = None , shuffle = False : 섞어서 제출하는 것 방지

result = model.predict(test_generator)



In [ ]:
sub = pd.read_csv("/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv")
sub.head()

In [ ]:
sub["id"] = test_generator.filenames

sub["id"] = sub["id"].apply(lambda x : x.split("/")[1].split(".")[0])

sub["label"] = result[:,1]

sub.head()

In [ ]:
sub.head(50)

#7234
#8775
Image.open("test/7234.jpg")

In [ ]:
sub.to_csv("sub.csv", index=False)